# Dataset

In [ ]:
from models.csv_loader import CSVLoader
from models.products.product_registry import ProductRegistry
from models.products.product_mapping_row import ProductMappingRow
from models.products.product_row import ProductRow

product_registry = ProductRegistry(CSVLoader(ProductRow).read(), CSVLoader(ProductMappingRow).read())

In [ ]:
from models.users.user_registry import UserRegistry
from models.users.user_mapping_row import UserMappingRow
from models.users.user_row import UserRow

user_registry = UserRegistry(CSVLoader(UserRow).read(), CSVLoader(UserMappingRow).read())

In [ ]:
from models.ratings.rating_registry import RatingRegistry
from models.ratings.rating_row import RatingRow

rating_registry = RatingRegistry(CSVLoader(RatingRow).read(), user_registry, product_registry)

# Rec method

In [ ]:
from models.reco.reco_factory import RecoFactory
import os 
from paths import PATHS

    
user_recos = dict()
for json_file_name in os.listdir(PATHS["recommendations"]):
    user_id = int(json_file_name.split("_")[-1].split(".")[0])
    user_reco_path = os.path.join(PATHS["recommendations"], json_file_name)
    user_recos[user_id] = RecoFactory.from_file(user_reco_path)

In [ ]:
user_recos[33][0]

# Rec from Recommender

In [ ]:
from recommendation.recommenders.random_recommender import RandomRecommender
from recommendation.recommenders.ilp_recommender import ILPRecommender

recommender = RandomRecommender(product_registry, user_registry, rating_registry)
#recommender = ILPRecommender(product_registry, user_registry, rating_registry)

In [ ]:
from models.ratings.rating import Rating
from models.users.user import User
import time

target_user = user_registry.add_user(User(eid=0, uid=0, gender="F", age="18-24"))

movie1 = product_registry.find_by_pid(pid=2346) # Fish Called Wanda, A (1988)
movie2 = product_registry.find_by_pid(pid=1762) # Fast Times at Ridgemont High (1982)

rating_registry.add_rating(Rating(target_user, movie1, 4, int(time.time())))
rating_registry.add_rating(Rating(target_user, movie2, 3, int(time.time())))

In [ ]:
recommendation_paths = recommender.recommend(target_user)
recommendation_paths

# Explanation

In [ ]:
import dotenv

dotenv.load_dotenv()

In [ ]:
import os

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

llm = HuggingFaceEndpoint(
    # repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    **{
        "max_new_tokens": 512,
        "top_k": 50,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

template = """{context}

Explain to {user} why {product} was recommended to them.
You can assume that the system is using a simple collaborative filtering approach.
The explanation should be clear and concise, without any technical jargon.
It should also be written in a way that is easy to understand for a non-technical user.
The tone should be friendly and helpful but avoid greeting the user.
The length should be around 1-2 paragraphs.
You can use phrases like "we noticed", "we thought", "we hope" to make the explanation more conversational. 
You can also use phrases like "similar interests", "related product", "many users who liked" to make the explanation more relatable and easy to understand.
Put your response inside <explanation> for parsing."""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm

In [ ]:
from recommendation.registry_handler import RegistryHandler
from recommendation.explainers.llm_explainer import LLMExplainer


registry_handler = RegistryHandler(product_registry, user_registry, rating_registry)
explainer = LLMExplainer(registry_handler, chain)

In [ ]:
#path = user_recos[23][3]
path = recommendation_paths[0] 
continuation, _ = explainer.explain(path)

In [ ]:
print(continuation)

In [ ]:
continuation_without_name, _ = explainer.explain(path, ["name"])
print(continuation_without_name)